# Offer

In [1]:
import pandas as pd
import re
import numpy as np

# 엑셀 파일의 절대 경로 또는 주피터 노트북 기준 상대 경로를 지정합니다.
file_path = r"C:\Users\light\Desktop\CSD - Course Offer.xlsx"  # Windows 경로 예시

# 엑셀 파일을 DataFrame으로 불러옵니다.
offer = pd.read_excel(file_path)

offer.drop(columns=['Unnamed: 0'], inplace=True)
offer.drop(offer.index[1157], axis=0, inplace=True)
offer['Capacity'] = offer['Capacity'].astype(int)
offer['Min Per Session'] = offer['Min Per Session'].astype(int)

total_time = 134790

offer['Session'] = offer['Session'].str.upper()
offer['Lecturer'] = offer['Lecturer'].str.upper()
offer['CourseCode'] = offer['CourseCode'].str.upper()
offer['FacultyCode'] = offer['FacultyCode'].str.upper()

offer['Session'] = offer['Session'].str.replace(r'\(S\)', '', regex=True)

# "COMBINED TO /"를 복사하여 error 테이블에 저장. 이후 원본 데이터에서 삭제.
mask = offer['Session'].str.contains("COMBINED TO /", na=False)

# error 데이터프레임에 해당 행들을 복사하여 저장합니다.
error = offer[mask].copy()

# 원본 offer 데이터프레임에서 해당 행들을 삭제합니다.
offer.drop(offer[mask].index, inplace=True)

#"GROUP A", "GROUP B", "GROUP C" 등 해당하는 행을 찾고, Session 값을 "LECTURE"로 변경
offer.loc[offer['Session'].str.contains(r'^GROUP\s+[A-Z]$', na=False), 'Session'] = 'LECTURE'

group_cols = ['CourseCode', 'FacultyCode', 'Session', 'Capacity', 'Min Per Session', 'Lecturer']

# 그룹별로 중복된 행을 처리합니다.
for _, group in offer.groupby(group_cols):
    if len(group) > 1:  # 중복 그룹인 경우
        # 원래 Session 값을 앞뒤 공백 제거 후 사용합니다.
        orig_session = group.iloc[0]['Session'].strip()
        # Session 값이 숫자로 끝나는지 검사 (예: "TUTORIAL 1")
        m = re.search(r'^(.*?)(\d+)$', orig_session)
        if m:
            base = m.group(1).strip()  # 문자 부분 (예: "TUTORIAL")
            start_num = int(m.group(2))  # 기존에 붙은 숫자 (예: 1)
        else:
            base = orig_session
            start_num = 1
        
        # 그룹 내 각 행에 대해 순차적으로 번호를 붙여 Session 값을 변경합니다.
        for offset, idx in enumerate(group.index):
            new_session = f"{base} {start_num + offset}"
            offer.at[idx, 'Session'] = new_session

# (초기 전제: offer DataFrame과 error DataFrame이 존재함; error가 없으면 빈 DataFrame 생성)
if 'error' not in globals():
    error = pd.DataFrame()

### Step 1. offer에서 "COMBINED TO" 행 추출 → child_class로 저장, offer에서는 제거
child_class = offer[offer['Session'].str.contains('COMBINED TO', na=False)].copy()
offer.drop(child_class.index, inplace=True)

### Step 2. child_class의 Session 열에서 과목코드 추출
child_class['extracted_code'] = child_class['Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)

### Step 3. child_class 내에서 오류 및 정상/후손 분리

# (A) "자기 자신 참조" 조건: 추출한 코드가 해당 행의 CourseCode와 같으면 → 오류
self_ref = child_class['extracted_code'] == child_class['CourseCode']

# (B) 정상(child) 조건: 자기 자신 참조가 아니고, 추출한 코드가 offer의 CourseCode에 존재하는 경우
valid_child_mask = (~self_ref) & (child_class['extracted_code'].isin(offer['CourseCode']))

# (C) 후손(grandchild) 후보: 자기 자신 참조가 아니고, 추출한 코드가 offer에는 없지만, child_class의 CourseCode에는 존재하는 경우  
#     (즉, 부모(대상 CourseCode)가 offer에 없으므로 자식끼리 연결되어 있음을 의미)
valid_grand_mask = (~self_ref) & (~child_class['extracted_code'].isin(offer['CourseCode'])) & \
                    (child_class['extracted_code'].isin(child_class['CourseCode']))

# (D) 오류 조건:  
#     - 자기 자신 참조인 경우  
#     - 또는 추출한 코드가 offer에도 child_class에도 존재하지 않는 경우
error_mask = self_ref | ((~child_class['extracted_code'].isin(offer['CourseCode'])) & 
                           (~child_class['extracted_code'].isin(child_class['CourseCode'])))

# child_class 오류 처리: 추출
child_error = child_class[error_mask].copy()

# 정상 child_class (valid_child) 추출
valid_child = child_class[valid_child_mask].copy()

# 후보 후손(grandchild) 추출
candidate_grand = child_class[valid_grand_mask].copy()

### Step 4. grand_child_class 내에서 추가 오류 처리
# 오류 조건 (grandchild): 자기 자신 참조 → 오류
grand_self_ref = candidate_grand['extracted_code'] == candidate_grand['CourseCode']
error_grand = candidate_grand[grand_self_ref].copy()

# 최종 grand_child_class: 후보에서 자기 자신 참조 오류 제거
grand_child_class = candidate_grand[~grand_self_ref].copy()

# 최종 error: 기존 child 오류와 후손에서 발생한 오류를 모두 누적
error_df = pd.concat([child_error, error_grand], ignore_index=True)
error = pd.concat([error, error_df], ignore_index=True)

### 최종 정상 데이터
# child_class는 valid_child로 업데이트 (offer의 부모가 존재하는 경우)
child_class = valid_child.copy()
# grand_child_class는 위에서 구한 대로

# error DataFrame이 없으면 빈 DataFrame 생성
if 'error' not in globals():
    error = pd.DataFrame()

##############################################
# 교차검증 1: child_class에서 자기 자신 참조하는 행 처리
##############################################
child_self_ref_mask = (child_class['extracted_code'] == child_class['CourseCode'])
child_self_ref_errors = child_class[child_self_ref_mask].copy()
error = pd.concat([error, child_self_ref_errors], ignore_index=True)
child_class = child_class[~child_self_ref_mask].copy()

##############################################
# 교차검증 2: child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
##############################################
child_missing_parent_mask = ~child_class['extracted_code'].isin(offer['CourseCode'])
child_missing_parent_errors = child_class[child_missing_parent_mask].copy()
error = pd.concat([error, child_missing_parent_errors], ignore_index=True)
child_class = child_class[~child_missing_parent_mask].copy()

##############################################
# 교차검증 3: offer에서 자기 자신 참조하는 행 처리
# (offer에 COMBINED TO가 남아있다면 해당 행에 대해 extracted_code를 추출)
##############################################
offer_combined_mask = offer['Session'].str.contains('COMBINED TO', na=False)
if offer_combined_mask.any():
    offer.loc[offer_combined_mask, 'extracted_code'] = offer.loc[offer_combined_mask, 'Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)
    offer_self_ref_mask = (offer['extracted_code'] == offer['CourseCode'])
    offer_self_ref_errors = offer[offer_self_ref_mask].copy()
    error = pd.concat([error, offer_self_ref_errors], ignore_index=True)
    offer.drop(offer[offer_self_ref_mask].index, inplace=True)

##############################################
# 교차검증 4: grand_child_class에서 자기 자신 참조하는 행 처리
##############################################
grand_self_ref_mask = (grand_child_class['extracted_code'] == grand_child_class['CourseCode'])
grand_self_ref_errors = grand_child_class[grand_self_ref_mask].copy()
error = pd.concat([error, grand_self_ref_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_self_ref_mask].copy()

##############################################
# 교차검증 5: grand_child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
##############################################
grand_missing_parent_mask = ~grand_child_class['extracted_code'].isin(offer['CourseCode'])
grand_missing_parent_errors = grand_child_class[grand_missing_parent_mask].copy()
error = pd.concat([error, grand_missing_parent_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_missing_parent_mask].copy()

##############################################
# 교차검증 6: 부모(offer 또는 child_class)에서 드랍된 행과 연결된 자식(또는 손자) 행 처리
# 드랍된 행들의 CourseCode를 모아서, 이 코드를 extracted_code로 사용하는 행이 있다면 error로 처리
##############################################
# 지금까지 error에 추가된 행들의 CourseCode를 모읍니다.
dropped_codes = set(error['CourseCode'].unique())

# child_class에서 연결된 자식 행 검사
child_linked_mask = child_class['extracted_code'].isin(dropped_codes)
child_linked_errors = child_class[child_linked_mask].copy()
error = pd.concat([error, child_linked_errors], ignore_index=True)
child_class = child_class[~child_linked_mask].copy()

# grand_child_class에서 연결된 손자 행 검사
grand_linked_mask = grand_child_class['extracted_code'].isin(dropped_codes)
grand_linked_errors = grand_child_class[grand_linked_mask].copy()
error = pd.concat([error, grand_linked_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_linked_mask].copy()

# child_class의 각 행을 순회하며 조건에 맞는 offer 행의 Capacity에 child_class의 Capacity를 합산합니다.
for idx, child_row in child_class.iterrows():
    # 조건: offer의 CourseCode, Min Per Session, Lecturer가 child_row의 extracted_code, Min Per Session, Lecturer와 일치
    mask = (
        (offer['CourseCode'] == child_row['extracted_code']) &
        (offer['Min Per Session'] == child_row['Min Per Session']) &
        (offer['Lecturer'] == child_row['Lecturer'])
    )
    # 일치하는 행이 있다면, offer의 Capacity에 child_row의 Capacity를 더함
    if mask.any():
        offer.loc[mask, 'Capacity'] += child_row['Capacity']

# 기존 조건
conditions = [
    offer['Session'].str.contains('LECTURE & TUTORIAL', na=False, case=False),
    offer['Session'].str.contains('TUTORIAL', na=False, case=False),
    offer['Session'].str.contains('LECTURE', na=False, case=False),
    offer['Session'].str.contains('LAB', na=False, case=False),
    offer['Session'].str.contains('PBL 1', na=False, case=False),
    offer['Session'].str.contains('HAND DRAWING|CAD DRAWING 2|CAD DRAWING 1', na=False, case=False),
    offer['Session'].str.contains('KITCHEN', na=False, case=False),
    offer['Session'].str.contains('OPTOM CLINIC|SOO - EXAM CLINIC', na=False, case=False)
]

# 기존 선택지 + 새 선택지
choices = [
    'GENERAL',
    'TUTORIAL',
    'LECTURE',
    'LAB',
    'PBL',
    'ART',
    'KITCHEN',
    'LECTURE'
]

# 조건을 적용하여 'Category' 열 생성
offer['Category'] = np.select(conditions, choices, default=np.nan)

offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
6,AB443,SABE,LECTURE,25,180,NURUL ANIDA MOHAMAD,LECTURE
7,AB516,SABE,LECTURE FATHER CLASS,50,450,LIM KER CHWING,LECTURE
8,AB533,SABE,LECTURE FATHER CLASS,40,180,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,LECTURE
10,AB618,SABE,LECTURE 2,20,240,NURUL ANIDA MOHAMAD,LECTURE
11,AB618,SABE,LECTURE 1,20,210,NURUL ANIDA MOHAMAD,LECTURE
...,...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE 1,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1153,SP206,FOSSLA,LECTURE 2,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK,LECTURE
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK,LECTURE


In [248]:
import pandas as pd

# 엑셀 파일의 절대 경로 또는 주피터 노트북 기준 상대 경로를 지정합니다.
file_path = r"C:\Users\light\Desktop\CSD - Course Offer.xlsx"  # Windows 경로 예시

# 엑셀 파일을 DataFrame으로 불러옵니다.
offer = pd.read_excel(file_path)

offer.drop(columns=['Unnamed: 0'], inplace=True)
offer.drop(offer.index[1157], axis=0, inplace=True)
offer['Capacity'] = offer['Capacity'].astype(int)
offer['Min Per Session'] = offer['Min Per Session'].astype(int)

total_time = 134790

offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,Lecture Combined To BEI1083/Lecture,5,180,Nurul Anida Mohamad
1,AB316,SABE,Lecture Combined To BEI2017/Lecture,5,450,Dr. Wong Leong Yee
2,AB343,SABE,Lecture Combined To BEI2033/Lecture,5,180,Humanson Gimfil
3,AB416,SABE,Lecture 1 Combined To BEI2067/Lecture 1,10,210,Asst. Prof. IDr Baizura Hanim Bt Bidin
4,AB416,SABE,Lecture 2 Combined To BEI2067/Lecture 2,10,240,Asst. Prof. IDr Baizura Hanim Bt Bidin
...,...,...,...,...,...,...
1152,SP206,FOSSLA,Lecture (S),20,180,Nurul Hidayah Bt Mohd Sa'at
1153,SP206,FOSSLA,Lecture (S),20,180,Nurul Hidayah Bt Mohd Sa'at
1154,SP208,FOSSLA,Lecture (S),5,120,Arman Imran Ashok
1155,SP208,FOSSLA,Lecture (S),5,180,Arman Imran Ashok


In [249]:
#전체 데이터를 대문자로 바꾸는 코드
offer['Session'] = offer['Session'].str.upper()
offer['Lecturer'] = offer['Lecturer'].str.upper()
offer['CourseCode'] = offer['CourseCode'].str.upper()
offer['FacultyCode'] = offer['FacultyCode'].str.upper()

offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE (S),20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE (S),20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE (S),5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE (S),5,180,ARMAN IMRAN ASHOK


In [250]:
#전체 데이터에서 (s)를 삭제하는 코드
offer['Session'] = offer['Session'].str.replace(r'\(S\)', '', regex=True)
offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK


In [251]:
# "COMBINED TO /"를 복사하여 error 테이블에 저장. 이후 원본 데이터에서 삭제.
mask = offer['Session'].str.contains("COMBINED TO /", na=False)

# error 데이터프레임에 해당 행들을 복사하여 저장합니다.
error = offer[mask].copy()

# 원본 offer 데이터프레임에서 해당 행들을 삭제합니다.
offer.drop(offer[mask].index, inplace=True)

error

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
80,BAC1014,FAS,COMBINED TO /,5,180,NaN
162,BBA1134,FBM,COMBINED TO /,20,180,NaN
863,EA103,FETBE,COMBINED TO /,2,60,NaN
864,EA103,FETBE,COMBINED TO /,2,60,NaN
1050,MS201,FAS,COMBINED TO /,10,120,NaN
1051,MS201,FAS,COMBINED TO /,10,60,NaN
1052,MS201,FAS,COMBINED TO /,10,60,NaN
1053,MS201,FAS,COMBINED TO /,10,120,NaN


In [252]:
offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE,20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK


In [253]:
#"GROUP A", "GROUP B", "GROUP C" 등 해당하는 행을 찾고, Session 값을 "LECTURE"로 변경
offer.loc[offer['Session'].str.contains(r'^GROUP\s+[A-Z]$', na=False), 'Session'] = 'LECTURE'

# 결과 확인
offer.loc[[1027]]


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
1027,MPU3221,FOSSLA,LECTURE,300,60,DR. SITI NUR AAFIFAH HASHIM


In [254]:
#중복된 행들에 숫자를 붙여서 다른 수업임을 명시. 이때 이미 숫자가 붙어있는상태로 중복이라면, 해당 숫자를 기준으로 중복을 처리함.
import re

group_cols = ['CourseCode', 'FacultyCode', 'Session', 'Capacity', 'Min Per Session', 'Lecturer']

# 그룹별로 중복된 행을 처리합니다.
for _, group in offer.groupby(group_cols):
    if len(group) > 1:  # 중복 그룹인 경우
        # 원래 Session 값을 앞뒤 공백 제거 후 사용합니다.
        orig_session = group.iloc[0]['Session'].strip()
        # Session 값이 숫자로 끝나는지 검사 (예: "TUTORIAL 1")
        m = re.search(r'^(.*?)(\d+)$', orig_session)
        if m:
            base = m.group(1).strip()  # 문자 부분 (예: "TUTORIAL")
            start_num = int(m.group(2))  # 기존에 붙은 숫자 (예: 1)
        else:
            base = orig_session
            start_num = 1
        
        # 그룹 내 각 행에 대해 순차적으로 번호를 붙여 Session 값을 변경합니다.
        for offset, idx in enumerate(group.index):
            new_session = f"{base} {start_num + offset}"
            offer.at[idx, 'Session'] = new_session

offer.loc[[166]]

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
166,BBA1703,FBM,TUTORIAL 2,40,60,BRAHAM RAHUL RAM A/L JAMNADAS


In [255]:
import pandas as pd

# (초기 전제: offer DataFrame과 error DataFrame이 존재함; error가 없으면 빈 DataFrame 생성)
if 'error' not in globals():
    error = pd.DataFrame()

### Step 1. offer에서 "COMBINED TO" 행 추출 → child_class로 저장, offer에서는 제거
child_class = offer[offer['Session'].str.contains('COMBINED TO', na=False)].copy()
offer.drop(child_class.index, inplace=True)

### Step 2. child_class의 Session 열에서 과목코드 추출
child_class['extracted_code'] = child_class['Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)

### Step 3. child_class 내에서 오류 및 정상/후손 분리

# (A) "자기 자신 참조" 조건: 추출한 코드가 해당 행의 CourseCode와 같으면 → 오류
self_ref = child_class['extracted_code'] == child_class['CourseCode']

# (B) 정상(child) 조건: 자기 자신 참조가 아니고, 추출한 코드가 offer의 CourseCode에 존재하는 경우
valid_child_mask = (~self_ref) & (child_class['extracted_code'].isin(offer['CourseCode']))

# (C) 후손(grandchild) 후보: 자기 자신 참조가 아니고, 추출한 코드가 offer에는 없지만, child_class의 CourseCode에는 존재하는 경우  
#     (즉, 부모(대상 CourseCode)가 offer에 없으므로 자식끼리 연결되어 있음을 의미)
valid_grand_mask = (~self_ref) & (~child_class['extracted_code'].isin(offer['CourseCode'])) & \
                    (child_class['extracted_code'].isin(child_class['CourseCode']))

# (D) 오류 조건:  
#     - 자기 자신 참조인 경우  
#     - 또는 추출한 코드가 offer에도 child_class에도 존재하지 않는 경우
error_mask = self_ref | ((~child_class['extracted_code'].isin(offer['CourseCode'])) & 
                           (~child_class['extracted_code'].isin(child_class['CourseCode'])))

# child_class 오류 처리: 추출
child_error = child_class[error_mask].copy()

# 정상 child_class (valid_child) 추출
valid_child = child_class[valid_child_mask].copy()

# 후보 후손(grandchild) 추출
candidate_grand = child_class[valid_grand_mask].copy()

### Step 4. grand_child_class 내에서 추가 오류 처리
# 오류 조건 (grandchild): 자기 자신 참조 → 오류
grand_self_ref = candidate_grand['extracted_code'] == candidate_grand['CourseCode']
error_grand = candidate_grand[grand_self_ref].copy()

# 최종 grand_child_class: 후보에서 자기 자신 참조 오류 제거
grand_child_class = candidate_grand[~grand_self_ref].copy()

# 최종 error: 기존 child 오류와 후손에서 발생한 오류를 모두 누적
error_df = pd.concat([child_error, error_grand], ignore_index=True)
error = pd.concat([error, error_df], ignore_index=True)

### 최종 정상 데이터
# child_class는 valid_child로 업데이트 (offer의 부모가 존재하는 경우)
child_class = valid_child.copy()
# grand_child_class는 위에서 구한 대로

In [256]:
import pandas as pd

# error DataFrame이 없으면 빈 DataFrame 생성
if 'error' not in globals():
    error = pd.DataFrame()

##############################################
# 교차검증 1: child_class에서 자기 자신 참조하는 행 처리
##############################################
child_self_ref_mask = (child_class['extracted_code'] == child_class['CourseCode'])
child_self_ref_errors = child_class[child_self_ref_mask].copy()
error = pd.concat([error, child_self_ref_errors], ignore_index=True)
child_class = child_class[~child_self_ref_mask].copy()

##############################################
# 교차검증 2: child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
##############################################
child_missing_parent_mask = ~child_class['extracted_code'].isin(offer['CourseCode'])
child_missing_parent_errors = child_class[child_missing_parent_mask].copy()
error = pd.concat([error, child_missing_parent_errors], ignore_index=True)
child_class = child_class[~child_missing_parent_mask].copy()

##############################################
# 교차검증 3: offer에서 자기 자신 참조하는 행 처리
# (offer에 COMBINED TO가 남아있다면 해당 행에 대해 extracted_code를 추출)
##############################################
offer_combined_mask = offer['Session'].str.contains('COMBINED TO', na=False)
if offer_combined_mask.any():
    offer.loc[offer_combined_mask, 'extracted_code'] = offer.loc[offer_combined_mask, 'Session'].str.extract(r'COMBINED TO\s+([A-Z0-9]+)', expand=False)
    offer_self_ref_mask = (offer['extracted_code'] == offer['CourseCode'])
    offer_self_ref_errors = offer[offer_self_ref_mask].copy()
    error = pd.concat([error, offer_self_ref_errors], ignore_index=True)
    offer.drop(offer[offer_self_ref_mask].index, inplace=True)

##############################################
# 교차검증 4: grand_child_class에서 자기 자신 참조하는 행 처리
##############################################
grand_self_ref_mask = (grand_child_class['extracted_code'] == grand_child_class['CourseCode'])
grand_self_ref_errors = grand_child_class[grand_self_ref_mask].copy()
error = pd.concat([error, grand_self_ref_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_self_ref_mask].copy()

##############################################
# 교차검증 5: grand_child_class에서 추출한 과목코드가 offer의 CourseCode에 없음(부모 없음)
##############################################
grand_missing_parent_mask = ~grand_child_class['extracted_code'].isin(offer['CourseCode'])
grand_missing_parent_errors = grand_child_class[grand_missing_parent_mask].copy()
error = pd.concat([error, grand_missing_parent_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_missing_parent_mask].copy()

##############################################
# 교차검증 6: 부모(offer 또는 child_class)에서 드랍된 행과 연결된 자식(또는 손자) 행 처리
# 드랍된 행들의 CourseCode를 모아서, 이 코드를 extracted_code로 사용하는 행이 있다면 error로 처리
##############################################
# 지금까지 error에 추가된 행들의 CourseCode를 모읍니다.
dropped_codes = set(error['CourseCode'].unique())

# child_class에서 연결된 자식 행 검사
child_linked_mask = child_class['extracted_code'].isin(dropped_codes)
child_linked_errors = child_class[child_linked_mask].copy()
error = pd.concat([error, child_linked_errors], ignore_index=True)
child_class = child_class[~child_linked_mask].copy()

# grand_child_class에서 연결된 손자 행 검사
grand_linked_mask = grand_child_class['extracted_code'].isin(dropped_codes)
grand_linked_errors = grand_child_class[grand_linked_mask].copy()
error = pd.concat([error, grand_linked_errors], ignore_index=True)
grand_child_class = grand_child_class[~grand_linked_mask].copy()

##############################################
# 최종 결과 출력 (각 DataFrame의 shape 등 확인)
##############################################
print("offer 남은 행:", offer.shape)
print("child_class 남은 행:", child_class.shape)
print("grand_child_class 남은 행:", grand_child_class.shape)
print("error 행:", error.shape)


offer 남은 행: (808, 6)
child_class 남은 행: (183, 7)
grand_child_class 남은 행: (0, 7)
error 행: (166, 7)


In [257]:
child_class

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,extracted_code
0,AB243,SABE,LECTURE COMBINED TO BEI1083/LECTURE,5,180,NURUL ANIDA MOHAMAD,BEI1083
1,AB316,SABE,LECTURE COMBINED TO BEI2017/LECTURE,5,450,DR. WONG LEONG YEE,BEI2017
2,AB343,SABE,LECTURE COMBINED TO BEI2033/LECTURE,5,180,HUMANSON GIMFIL,BEI2033
3,AB416,SABE,LECTURE 1 COMBINED TO BEI2067/LECTURE 1,10,210,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,BEI2067
4,AB416,SABE,LECTURE 2 COMBINED TO BEI2067/LECTURE 2,10,240,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,BEI2067
...,...,...,...,...,...,...,...
1049,MS201,FAS,LECTURE 2 COMBINED TO BAA2023/LECTURE 2,10,60,ASST. PROF. DR. CHEW LI LEE,BAA2023
1054,MST1014B,FOSSLA,LECTURE COMBINED TO PST1014B/LECTURE 1,10,240,ASST. PROF. DR. ZAIDA MUSTAFA,PST1014B
1055,MST1014B,FOSSLA,LECTURE COMBINED TO PST1014B/LECTURE 2,10,240,ASST. PROF. DR. ZAIDA MUSTAFA,PST1014B
1056,MST1024B,FOSSLA,LECTURE COMBINED TO PST1024B/LECTURE 1,10,240,ASST. PROF. DR. SIMRANJEET KAUR JUDGE A/P CHAR...,PST1024B


In [258]:
# child_class의 각 행을 순회하며 조건에 맞는 offer 행의 Capacity에 child_class의 Capacity를 합산합니다.
for idx, child_row in child_class.iterrows():
    # 조건: offer의 CourseCode, Min Per Session, Lecturer가 child_row의 extracted_code, Min Per Session, Lecturer와 일치
    mask = (
        (offer['CourseCode'] == child_row['extracted_code']) &
        (offer['Min Per Session'] == child_row['Min Per Session']) &
        (offer['Lecturer'] == child_row['Lecturer'])
    )
    # 일치하는 행이 있다면, offer의 Capacity에 child_row의 Capacity를 더함
    if mask.any():
        offer.loc[mask, 'Capacity'] += child_row['Capacity']

# 결과 확인: 업데이트된 offer 데이터프레임의 Capacity 값
offer

,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer
6,AB443,SABE,LECTURE,25,180,NURUL ANIDA MOHAMAD
7,AB516,SABE,LECTURE FATHER CLASS,50,450,LIM KER CHWING
8,AB533,SABE,LECTURE FATHER CLASS,40,180,ASST. PROF. IDR BAIZURA HANIM BT BIDIN
10,AB618,SABE,LECTURE 2,20,240,NURUL ANIDA MOHAMAD
11,AB618,SABE,LECTURE 1,20,210,NURUL ANIDA MOHAMAD
...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE 1,20,180,NURUL HIDAYAH BT MOHD SA'AT
1153,SP206,FOSSLA,LECTURE 2,20,180,NURUL HIDAYAH BT MOHD SA'AT
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK


In [302]:
import numpy as np

# 기존 조건
conditions = [
    offer['Session'].str.contains('LECTURE & TUTORIAL', na=False, case=False),
    offer['Session'].str.contains('TUTORIAL', na=False, case=False),
    offer['Session'].str.contains('LECTURE', na=False, case=False),
    offer['Session'].str.contains('LAB', na=False, case=False),
    offer['Session'].str.contains('PBL 1', na=False, case=False),
    offer['Session'].str.contains('HAND DRAWING|CAD DRAWING 2|CAD DRAWING 1', na=False, case=False),
    offer['Session'].str.contains('KITCHEN', na=False, case=False),
    offer['Session'].str.contains('OPTOM CLINIC|SOO - EXAM CLINIC', na=False, case=False)
]

# 기존 선택지 + 새 선택지
choices = [
    'GENERAL',
    'TUTORIAL',
    'LECTURE',
    'LAB',
    'PBL',
    'ART',
    'KITCHEN',
    'LECTURE'
]

# 조건을 적용하여 'Category' 열 생성
offer['Category'] = np.select(conditions, choices, default=np.nan)

offer


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
6,AB443,SABE,LECTURE,25,180,NURUL ANIDA MOHAMAD,LECTURE
7,AB516,SABE,LECTURE FATHER CLASS,50,450,LIM KER CHWING,LECTURE
8,AB533,SABE,LECTURE FATHER CLASS,40,180,ASST. PROF. IDR BAIZURA HANIM BT BIDIN,LECTURE
10,AB618,SABE,LECTURE 2,20,240,NURUL ANIDA MOHAMAD,LECTURE
11,AB618,SABE,LECTURE 1,20,210,NURUL ANIDA MOHAMAD,LECTURE
...,...,...,...,...,...,...,...
1152,SP206,FOSSLA,LECTURE 1,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1153,SP206,FOSSLA,LECTURE 2,20,180,NURUL HIDAYAH BT MOHD SA'AT,LECTURE
1154,SP208,FOSSLA,LECTURE,5,120,ARMAN IMRAN ASHOK,LECTURE
1155,SP208,FOSSLA,LECTURE,5,180,ARMAN IMRAN ASHOK,LECTURE


In [303]:
result = offer[offer['CourseCode'].str.lower() == 'bp132']
result


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
541,BP132,FPS,LECTURE 1,60,60,PROF. TS. DR. LEE MING TATT,LECTURE
542,BP132,FPS,LECTURE 2,60,60,PROF. TS. DR. LEE MING TATT,LECTURE
543,BP132,FPS,LECTURE 3,60,60,PROF. TS. DR. LEE MING TATT,LECTURE
544,BP132,FPS,TUTORIAL 1,60,60,PROF. TS. DR. LEE MING TATT,TUTORIAL
545,BP132,FPS,LECTURE & TUTORIAL,60,60,PROF. TS. DR. LEE MING TATT,GENERAL


In [304]:
mask = ~offer['Category'].str.contains('TUTORIAL|LECTURE|LAB', na=False)
offer[mask]


,CourseCode,FacultyCode,Session,Capacity,Min Per Session,Lecturer,Category
537,BP112,FPS,LECTURE & TUTORIAL,140,60,PROFESSOR DR. SAAD TAYYAB,GENERAL
545,BP132,FPS,LECTURE & TUTORIAL,60,60,PROF. TS. DR. LEE MING TATT,GENERAL
548,BP172,FPS,LECTURE & TUTORIAL,60,60,VITHYAH A/P NADARAJA,GENERAL
568,BP266,FPS,LECTURE & TUTORIAL,60,60,SABREEN YOUSIF MOHAMMED ALHASSAN NASR,GENERAL
653,BPC2122,FPS,LECTURE & TUTORIAL,20,60,ASSOCIATE PROFESSOR DR. MAI CHUN WAI,GENERAL
839,DEX1034,FETBE,HAND DRAWING,15,120,TS. AMAR RIDZUAN ABD HAMID,ART
841,DEX1034,FETBE,CAD DRAWING 2,15,120,TS. MUHAMAD FALIQ MOHAMAD NAZER,ART
842,DEX1034,FETBE,CAD DRAWING 1,15,120,TS. MUHAMAD FALIQ MOHAMAD NAZER,ART
853,DPB1014,FHTM,KITCHEN PRACTICAL,10,360,SH MARIA SAHILA BT SYED ALI HASSAN,KITCHEN
854,DPB1024,FHTM,LECTURE & TUTORIAL,10,180,NURSYAFIQAH RAMLI,GENERAL


# classroom

In [22]:
import pandas as pd

# 엑셀 파일의 절대 경로 또는 주피터 노트북 기준 상대 경로를 지정합니다.
file_path = r"C:\Users\light\Desktop\CSD - Resource Room.xlsx" # Windows 경로 예시

# 엑셀 파일을 DataFrame으로 불러옵니다.
room = pd.read_excel(file_path)

# DataFrame의 상위 5개 행을 출력하여 파일 내용을 확인합니다.
room

,Campus,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Workshop,Resource Status
0,Kuala Lumpur Campus,C209,C209 - [C209],102.0,Y,Y,N,Y,Active
1,Kuala Lumpur Campus,C403,C403 - [C403],50.0,Y,Y,N,N,Active
2,Kuala Lumpur Campus,C405,C405 - [C405],135.0,Y,Y,N,N,Active
3,Kuala Lumpur Campus,C401,C401 - [C401],40.0,Y,Y,N,N,Active
4,Kuala Lumpur Campus,C407,C407 - [C407],100.0,Y,Y,N,N,Active
...,...,...,...,...,...,...,...,...,...
236,Kuala Lumpur Campus,Civil Workshop,"Civil Workshop, Level 4 Block E",30.0,N,N,Y,NaN,Active
237,Kuala Lumpur Campus,C480 - FETBE Block C,"Robotic Lab (Block C, C480)",30.0,Y,Y,NaN,NaN,Active
238,Kuala Lumpur Campus,FASMeetingRoom1,FAS Meeting Room 1,10.0,Y,Y,N,NaN,Active
239,Kuala Lumpur Campus,FASMeetingRoom2,FAS Meeting Room 2,10.0,N,N,N,NaN,Active


In [23]:
room = room.drop('Campus', axis=1)
room

,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Workshop,Resource Status
0,C209,C209 - [C209],102.0,Y,Y,N,Y,Active
1,C403,C403 - [C403],50.0,Y,Y,N,N,Active
2,C405,C405 - [C405],135.0,Y,Y,N,N,Active
3,C401,C401 - [C401],40.0,Y,Y,N,N,Active
4,C407,C407 - [C407],100.0,Y,Y,N,N,Active
...,...,...,...,...,...,...,...,...
236,Civil Workshop,"Civil Workshop, Level 4 Block E",30.0,N,N,Y,NaN,Active
237,C480 - FETBE Block C,"Robotic Lab (Block C, C480)",30.0,Y,Y,NaN,NaN,Active
238,FASMeetingRoom1,FAS Meeting Room 1,10.0,Y,Y,N,NaN,Active
239,FASMeetingRoom2,FAS Meeting Room 2,10.0,N,N,N,NaN,Active


In [24]:
room = room.drop('Workshop', axis=1)
room

,Resource Code,Description,Capacity,Lecture,Tutorial,Lab,Resource Status
0,C209,C209 - [C209],102.0,Y,Y,N,Active
1,C403,C403 - [C403],50.0,Y,Y,N,Active
2,C405,C405 - [C405],135.0,Y,Y,N,Active
3,C401,C401 - [C401],40.0,Y,Y,N,Active
4,C407,C407 - [C407],100.0,Y,Y,N,Active
...,...,...,...,...,...,...,...
236,Civil Workshop,"Civil Workshop, Level 4 Block E",30.0,N,N,Y,Active
237,C480 - FETBE Block C,"Robotic Lab (Block C, C480)",30.0,Y,Y,NaN,Active
238,FASMeetingRoom1,FAS Meeting Room 1,10.0,Y,Y,N,Active
239,FASMeetingRoom2,FAS Meeting Room 2,10.0,N,N,N,Active
